Synthetic data, one warehouse lost demand

In [17]:
%load_ext autoreload
%autoreload 2

import analysis.ray_results_interpreter as rri
from importlib import reload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/50"
}
conditions = {'context': [0, 1, 256]}
df = results_interpretor.make_table(paths, conditions)
df.rename(columns={'context': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)
print(df.to_string(index=False))

 # of stores  context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3             0         0.0100    5.670470  5.657994   5.663223
           3             1         0.0100    5.613812  5.608477   5.609850
           3           256         0.0100    5.616237  5.611364   5.612627
           5             0         0.0010    5.312334  5.292432   5.301958
           5             1         0.0100    5.256996  5.235224   5.247245
           5           256         0.0001    5.249202  5.237397   5.249729
          10             0         0.0100    5.773343  5.808772   5.776743
          10             1         0.0100    5.712263  5.755829   5.720371
          10           256         0.0100    5.731197  5.761904   5.729029
          20             0         0.0010    5.902905  5.892253   5.871639
          20             1         0.0010    5.850531  5.839253   5.818287
          20           256         0.0100    5.857646  5.844117   5.820630
          30             

In [32]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/3",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/20",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/50"
}

results_interpretor = rri.RayResultsinterpreter()
conditions = {'for_all_networks': [1, 16, 64]}
df = results_interpretor.make_table(paths, conditions)
df.rename(columns={'for_all_networks': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)
print(df.to_string(index=False))

 # of stores  context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3             1           0.01    5.623115  5.613710   5.616462
           3            16           0.01    5.618455  5.607520   5.609473
           3            64           0.01    5.616272  5.608735   5.608330
          10             1           0.01    5.805394  5.818976   5.786214
          10            16           0.01    5.708765  5.751087   5.717375
          10            64           0.01    5.712751  5.752891   5.718822
          20             1           0.01   15.638020 15.544399  15.692766
          20            16           0.01    5.841372  5.834482   5.812328
          50             1           0.01   11.232640 11.216736  11.230151
          50            16           0.01   13.213552 13.178728  14.110262
          50            64           0.01   22.270393 22.313019  30.210777


In [33]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/3",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/20",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/50"
}
results_interpretor = rri.RayResultsinterpreter()
conditions = {'for_all_networks': [1, 16, 64]}
df = results_interpretor.make_table(paths, conditions)
df.rename(columns={'for_all_networks': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)
print(df.to_string(index=False))

 # of stores  context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3             1          0.010    5.623565  5.616145   5.620568
           3            16          0.010    5.613380  5.608589   5.608596
           3            64          0.010    5.614989  5.612327   5.613634
          10             1          0.001    5.728411  5.772287   5.738262
          10            16          0.010    5.711121  5.752939   5.717505
          10            64          0.010    5.771418  5.807780   5.775408
          20             1          0.001    5.922441  5.902557   5.882965
          20            16          0.001    6.475514  6.416679   6.434179
          20            64          0.010    5.905616  5.888665   5.869436
          50             1          0.010    8.102721  8.089805  10.459843
          50            16          0.001    6.631942  6.602454   6.568062
          50            64          0.001    5.358920  5.338118   5.357194


In [14]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/50"
}
df_ctx = make_the_result_table(paths, [0, 1, 256])
df_ctx.insert(1, 'Architecture Class', "Symmetry_Aware")
df_ctx = df_ctx.loc[df_ctx.groupby(['# of stores'])['Dev Loss'].idxmin()]

paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/50"
}
df_gnn = make_the_result_table(paths, [1, 2, 4, 8, 16, 32, 64, 128])
df_gnn.insert(1, 'Architecture Class', "GNN")
df_gnn = df_gnn.loc[df_gnn.groupby(['# of stores'])['Dev Loss'].idxmin()]

paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/50"
}
df_gnn_mp = make_the_result_table(paths, [1, 2, 4, 8, 16, 32, 64, 128])
df_gnn_mp.insert(1, 'Architecture Class', "GNN Message Passing")
df_gnn_mp = df_gnn_mp.loc[df_gnn_mp.groupby(['# of stores'])['Dev Loss'].idxmin()]

vanilla = [
    {
                "# of stores": 3,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.610,
                "Dev Loss": 5.610,
                "Test Loss": 5.610,
            },
    {
                "# of stores": 5,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.250,
                "Dev Loss": 5.250,
                "Test Loss": 5.240,
            },
    {
                "# of stores": 10,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.720,
                "Dev Loss": 5.740,
                "Test Loss": 5.720,
            },
    {
                "# of stores": 20,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.850,
                "Dev Loss": 5.870,
                "Test Loss": 5.850,
            },
    {
                "# of stores": 30,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.580,
                "Dev Loss": 5.60,
                "Test Loss": 5.59,
            },
    {
                "# of stores": 50,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.410,
                "Dev Loss": 5.400,
                "Test Loss": 5.420,
            },
]
df_vanilla = pd.DataFrame(vanilla)

df = pd.concat([df_ctx, df_gnn, df_gnn_mp, df_vanilla])
architecture_order = ['Symmetry_Aware', 'GNN', 'GNN Message Passing', 'Vanilla']
df['Architecture Class'] = pd.Categorical(df['Architecture Class'], categories=architecture_order, ordered=True)
df.sort_values(by=['# of stores', 'Architecture Class'], inplace=True)
print(df.to_string(index=False))

Error processing files in /user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/50/run_2024-07-30_04-49-05/run_93d51_00022_22_for_all_networks=64,learning_rate=0.0001,samples=0_2024-07-30_04-49-06: Expecting value: line 1 column 1 (char 0)
Error processing files in /user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/50/run_2024-07-30_04-49-05/run_93d51_00002_2_for_all_networks=4,learning_rate=0.0100,samples=0_2024-07-30_04-49-06: No columns to parse from file
 # of stores  Architecture Class context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3      Symmetry_Aware            1          0.010    5.613812  5.608477   5.609850
           3                 GNN            8          0.010    5.615963  5.607577   5.610156
           3 GNN Message Passing            4          0.010    5.615473  5.608413   5.608291
           3             Vanilla         None            NaN    5.610000  5.610000   5.610000
           5      Symmetry_Aware            1          0

Synthetic data, Transshipment

In [84]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/transshipment/3",
    5: "/user/ml4723/Prj/NIC/ray_results/transshipment/5",
    10: "/user/ml4723/Prj/NIC/ray_results/transshipment/10",
}
df_sym = make_the_result_table(paths, [0, 1, 256])
df_sym.insert(1, 'Architecture Class', "Symmetry_Aware")

paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/3",
    5: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/5",
    10: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/10",
}
df_van = make_the_result_table(paths, [None])
df_van.insert(1, 'Architecture Class', "Vanilla")

lower_bound = [
    {
                "# of stores": 3,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 6.19,
            },
    {
                "# of stores": 5,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 5.75,
            },
    {
                "# of stores": 10,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 6.05,
            },
]
df_lower_bound = pd.DataFrame(lower_bound)

df = pd.concat([df_sym, df_van, df_lower_bound])
df.sort_values(by=['# of stores', 'context size'], inplace=True)
print(df.to_string(index=False))

 # of stores Architecture Class context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3     Symmetry_Aware            0         0.0100   15.786119 15.745235  38.493707
           3     Symmetry_Aware            1         0.0010    6.451584  6.442219   6.436544
           3     Symmetry_Aware          256         0.0100    6.215819  6.195688   6.190809
           3            Vanilla         None         0.0001    6.202121  6.195605   6.190790
           3        Lower bound         None            NaN         NaN       NaN   6.190000
           5     Symmetry_Aware            0         0.0010   14.106193 14.152811  34.056048
           5     Symmetry_Aware            1         0.0100    6.061970  6.045415   6.036085
           5     Symmetry_Aware          256         0.0010    5.751615  5.759176   5.751703
           5            Vanilla         None         0.0001    5.755991  5.759228   5.751869
           5        Lower bound         None            NaN         Na

One warehouse lost demand synthetic - different primitive setup

In [2]:
def make_the_result_table_diff_primitive(paths, context_sizes, warehouse_holding_costs, warehouse_lead_times, stores_correlations):
    # Placeholder for results
    results = []

    # Iterate over each path and context size, call the function, and store the top row
    for num_stores, path in paths.items():
        for context_size in context_sizes:
            df = create_results_table(path, context_size)
            if df is None:
                continue
            for warehouse_holding_cost in warehouse_holding_costs:
                df_hc = df[df['warehouse_holding_cost'] == warehouse_holding_cost]
                if df_hc.empty:
                    continue
                for warehouse_lead_time in warehouse_lead_times:
                    df_hl = df_hc[df_hc['warehouse_lead_time'] == warehouse_lead_time]
                    if df_hl.empty:
                        continue
                    for stores_correlation in stores_correlations:
                        df_sc = df_hl[df_hl['stores_correlation'] == stores_correlation]
                        if df_sc.empty:
                            continue
                        top_row = df_sc.iloc[0]
                        result_row = {
                            'warehouse_holding_cost': top_row['warehouse_holding_cost'],
                            'warehouse_lead_time': top_row['warehouse_lead_time'],
                            'stores_correlation': top_row['stores_correlation'],
                            "context size": context_size,
                            "Learning Rate": top_row['learning_rate'],
                            "Train Loss": top_row['train_loss(at best_dev)'],
                            "Dev Loss": top_row['best_dev_loss'],
                            "Test Loss": top_row['test_loss(at best_dev)'],
                            "path": top_row['path'],
                        }
                        results.append(result_row)
    result_df = pd.DataFrame(results)
    min_test_loss = result_df.groupby(['warehouse_holding_cost', 'warehouse_lead_time', 'stores_correlation'])['Test Loss'].transform('min')
    result_df['Test Gap %'] = ((result_df['Test Loss'] - min_test_loss) / min_test_loss) * 100
    result_df.sort_values(by=['warehouse_holding_cost', 'warehouse_lead_time', 'stores_correlation', 'context size'], inplace=True)
    return result_df

In [8]:
paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/diff_primitive/ctx'
}

# df = make_the_result_table_diff_primitive(paths, [0, 1, 16, 64], [0.7, 1.0, 1.3, 2.0], [2, 6], [-0.5, 0.0, 0.5])
df = make_the_result_table_diff_primitive(paths, [0, 1, 16, 64], [0.7, 1.0, 1.3, 2.0], [6], [0.5])
print(df.to_string(index=False))

 warehouse_holding_cost  warehouse_lead_time  stores_correlation  context size  Learning Rate  Train Loss  Dev Loss  Test Loss                                                                                                                                                                                      path  Test Gap %
                    0.7                    6                 0.5             0          0.010    5.913846  5.907117   5.913744 /user/ml4723/Prj/NIC/ray_results/diff_primitive/ctx/run_2024-07-31_02-41-23/run_e7229_00012_12_context=0,learning_rate=0.0100,samples=2,warehouse_holding_cost=0.7000_2024-07-31_02-41-23    1.509709
                    0.7                    6                 0.5             1          0.001    5.843374  5.837429   5.838852  /user/ml4723/Prj/NIC/ray_results/diff_primitive/ctx/run_2024-07-31_02-41-23/run_e7229_00005_5_context=1,learning_rate=0.0010,samples=1,warehouse_holding_cost=0.7000_2024-07-31_02-41-23    0.224195
                    0.7  